In [16]:
import polars as pl
from pathlib import *
import os

In [5]:
# Setting up paths

In [3]:
base = Path(".")
input = base / "data" / "input"
traits_path = input / "traits_no_measurement.tsv"
traits_df = pl.read_csv(str(traits_path), sep="\t")

traits_df_nullsrem = (
    traits_df
        .with_column(pl.all().fill_null(0))
        .filter(pl.col("Olga's comments")!="0")
) #removed nulls

In [14]:
comments = traits_df_nullsrem["Olga's comments"].to_list()
comments_low = [x.lower().strip() for x in comments]
comments_series = pl.Series(comments_low)

traits_grouped = traits_df_nullsrem.with_columns(pl.lit(comments_series).alias("comments")).drop("Olga's comments").groupby('comments').agg(pl.all())


comments,trait,trait_uri,studies,association_counts,papers
str,list[str],list[str],list[i64],list[i64],list[i64]
"""longevity?""","[""telomere length"", ""longevity"", ""aging""]","[""http://www.ebi.ac.uk/efo/EFO_0004505"", ""http://www.ebi.ac.uk/efo/EFO_0004300"", ""http://purl.obolibrary.org/obo/GO_0007568""]","[25, 22, 22]","[302, 217, 203]","[25, 22, 22]"
"""musculoskeleta...","[""bone fracture"", ""osteoarthritis"", ... ""Abnormality of bone mineral density""]","[""http://www.ebi.ac.uk/efo/EFO_0003931"", ""http://purl.obolibrary.org/obo/MONDO_0005178"", ... ""http://purl.obolibrary.org/obo/HP_0004348""]","[85, 60, ... 4]","[53, 99, ... 0]","[85, 60, ... 4]"
"""glucose homeos...","[""lean mass-adjusted fat body mass""]","[""http://www.ebi.ac.uk/efo/EFO_0009890""]",[3],[17],[3]
"""musculosceleta...","[""osteoarthritis, knee""]","[""http://www.ebi.ac.uk/efo/EFO_0004616""]",[21],[64],[21]
"""other""","[""age-related macular degeneration"", ""age at menopause"", ... ""response to mitochondrial complex I inhibitor""]","[""http://www.ebi.ac.uk/efo/EFO_0001365"", ""http://www.ebi.ac.uk/efo/EFO_0004704"", ... ""http://www.ebi.ac.uk/efo/EFO_0600033""]","[31, 22, ... 1]","[128, 327, ... 13]","[31, 22, ... 1]"
"""muskuloskeleta...","[""vitamin D deficiency""]","[""http://www.ebi.ac.uk/efo/EFO_0003762""]",[7],[30],[7]
"""glucose homeos...","[""diabetes mellitus""]","[""http://www.ebi.ac.uk/efo/EFO_0000400""]",[25],[6],[25]
"""mental""","[""Alzheimer disease"", ""Parkinson disease"", ... ""Alzheimer disease, Cognitive impairment""]","[""http://purl.obolibrary.org/obo/MONDO_0004975"", ""http://purl.obolibrary.org/obo/MONDO_0005180"", ... ""http://purl.obolibrary.org/obo/MONDO_0004975, http://purl.obolibrary.org/obo/HP_0100543""]","[89, 49, ... 1]","[669, 452, ... 9]","[89, 49, ... 1]"
"""metabolic heal...","[""body fat distribution""]","[""http://www.ebi.ac.uk/efo/EFO_0004341""]",[19],[526],[19]


In [21]:
columns = traits_grouped.drop(["comments"]).columns

traits = traits_grouped.explode(columns)
traits

comments,trait,trait_uri,studies,association_counts,papers
str,str,str,i64,i64,i64
"""longevity?""","""telomere lengt...","""http://www.ebi...",25,302,25
"""longevity?""","""longevity""","""http://www.ebi...",22,217,22
"""longevity?""","""aging""","""http://purl.ob...",22,203,22
"""musculoskeleta...","""bone fracture""","""http://www.ebi...",85,53,85
"""musculoskeleta...","""osteoarthritis...","""http://purl.ob...",60,99,60
"""musculoskeleta...","""bone density""","""http://www.ebi...",53,692,53
"""musculoskeleta...","""lean body mass...","""http://www.ebi...",45,523,45
"""musculoskeleta...","""osteoporosis""","""http://www.ebi...",33,45,33
"""musculoskeleta...","""heel bone mine...","""http://www.ebi...",25,4690,25


In [60]:
modules = set(comments_low)
df_list = []

dir_path = base / "data" / "interim" 

for item in modules:
     # creating directory
    if "/" in item:
        item = item.replace("/", ", ")
    if "?" in item:
        item = item.replace("?", "")
                            
    item_path = os.path.join(dir_path, item)
    os.makedirs(item_path,exist_ok=True)
    file_name = item + '.tsv'
    file_path = dir_path / item / file_name

    module_df = traits.filter(pl.col('comments')==item)

    module_df.write_csv(file = str(file_path), sep='\t')
